In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras as k

2024-03-05 23:13:23.080449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 23:13:23.080483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 23:13:23.081249: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 23:13:23.086040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 23:13:23.724807: W tensorflow/compiler/tf2

In [2]:
data = pd.read_json("data/sarcasm.json", lines=True)
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
data.dropna(axis=0, how='any')
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
data_train = data.iloc[0:int(len(data)*0.8)]
data_test = data.iloc[int(len(data)*0.8):]

X_train = data_train['headline'].values
Y_train = data_train['is_sarcastic'].values

X_test = data_test['headline'].values
Y_test = data_test['is_sarcastic'].values

In [5]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [6]:
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

max_length = max(max([len(sequence) for sequence in sequences_train]), max([len(sequence) for sequence in sequences_test]))

sequences_train = pad_sequences(sequences_train, maxlen=max_length, padding='post')
sequences_test = pad_sequences(sequences_test, maxlen=max_length, padding='post')

In [9]:
model = k.Sequential([
    k.layers.Embedding(input_length=max_length, input_dim= len(tokenizer.word_index), output_dim=100),
    k.layers.GlobalMaxPool1D(),
    k.layers.Dense(128, activation='relu'),
    k.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

In [10]:
model.fit(
    x= sequences_train,
    y= Y_train,
    batch_size= 500,
    epochs=25,
    validation_data=(sequences_test, Y_test),
    shuffle= True
)

Epoch 1/25
43/43 [==============================] - 4s 74ms/step - loss: 0.6716 - accuracy: 0.5540 - val_loss: 0.6265 - val_accuracy: 0.6176
Epoch 2/25
43/43 [==============================] - 2s 54ms/step - loss: 0.4934 - accuracy: 0.8154 - val_loss: 0.4086 - val_accuracy: 0.8319
Epoch 3/25
43/43 [==============================] - 2s 46ms/step - loss: 0.2610 - accuracy: 0.9078 - val_loss: 0.3348 - val_accuracy: 0.8529
Epoch 4/25
43/43 [==============================] - 1s 30ms/step - loss: 0.1332 - accuracy: 0.9598 - val_loss: 0.3309 - val_accuracy: 0.8555
Epoch 5/25
43/43 [==============================] - 1s 33ms/step - loss: 0.0646 - accuracy: 0.9847 - val_loss: 0.3460 - val_accuracy: 0.8568
Epoch 6/25
43/43 [==============================] - 1s 20ms/step - loss: 0.0298 - accuracy: 0.9953 - val_loss: 0.3700 - val_accuracy: 0.8559
Epoch 7/25
43/43 [==============================] - 1s 25ms/step - loss: 0.0143 - accuracy: 0.9990 - val_loss: 0.3903 - val_accuracy: 0.8572
Epoch 8/25
43

In [15]:
real_phrases = [
    "sunny is shining today",
    "so snowy today",
    "mom is going to shop",
    "father is going to shop",
    "Everyone has the right to be stupid. But you are abusing it."
]
real_phrases = tokenizer.texts_to_sequences(real_phrases)
real_phrases = pad_sequences(real_phrases, maxlen=max_length, padding='post')

In [16]:
model.predict(real_phrases)

1/1 [==============================] - 0s 13ms/step


array([[0.01039973],
       [0.46577522],
       [0.7108093 ],
       [0.8908721 ],
       [0.5655917 ]], dtype=float32)